# cs model v.00 (big balz)

In [10]:
# import main libs
import numpy as np
import pandas as pd

In [13]:
df = pd.read_csv(r'../tsla_stock_data.csv').drop([0,1])

In [16]:
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

In [33]:
# Convert Date column to datetime if not already
df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [34]:
df = df.sort_values(by="Date")

In [35]:
# 1️⃣ Daily Return
df['Daily_Return'] = (df['Close'] - df['Open']) / df['Open']

In [36]:
# 2️⃣ Price Momentum (Close Price Difference)
df['Price_Momentum'] = df['Close'].diff()

In [37]:
# 3️⃣ Volatility (High - Low range)
df['Volatility'] = (df['High'] - df['Low']) / df['Close']

In [38]:
# 4️⃣ Percentage Change from Previous Close
df['Pct_Change'] = df['Close'].pct_change() * 100

In [39]:
# 5️⃣ Simple Moving Averages (SMA)
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_10'] = df['Close'].rolling(window=10).mean()

In [40]:
# 6️⃣ Exponential Moving Averages (EMA)
df['EMA_5'] = df['Close'].ewm(span=5, adjust=False).mean()
df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()

In [41]:
# 7️⃣ Volume Change
df['Volume_Change'] = df['Volume'].diff()

In [42]:
# 8️⃣ Relative Strength Index (RSI)
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI_14'] = compute_rsi(df['Close'])

In [43]:
# Drop NaN values created by rolling calculations
df = df.dropna()

In [44]:
df.head()

,Date,Close,High,Low,Open,Volume,Daily_Return,Price_Momentum,Volatility,Pct_Change,SMA_5,SMA_10,EMA_5,EMA_10,Volume_Change,RSI_14
28,2025-01-24 19:30:00,406.850006,409.750000,406.799988,407.924988,4577294,-0.002635,-1.100006,0.007251,-0.269642,409.984003,411.059491,409.260902,410.665531,-804208.0,28.714975
29,2025-01-24 20:30:00,406.535004,407.850006,405.779999,406.925110,4577762,-0.000959,-0.315002,0.005092,-0.077425,408.589001,410.474991,408.352269,409.914526,468.0,28.255702
30,2025-01-27 14:30:00,402.274994,405.759888,393.393005,394.532013,20392868,0.019626,-4.260010,0.030742,-1.047883,406.764001,409.613489,406.326511,408.525520,15815106.0,23.940411
31,2025-01-27 15:30:00,399.279999,406.669891,398.619995,402.230011,7667701,-0.007334,-2.994995,0.020161,-0.744514,404.578003,408.492990,403.977674,406.844517,-12725167.0,22.144649
32,2025-01-27 16:30:00,397.154785,401.507599,395.050110,399.329987,5533368,-0.005447,-2.125214,0.016259,-0.532261,402.418958,406.971469,401.703377,405.082747,-2134333.0,22.389465


In [45]:
df.to_csv("tsla_stock_data_with_features.csv", index=False)